In [1]:
import pandas as pd
import numpy as np
from lxml import html
import requests
import os

In [2]:
# cwd = os.getcwd()  # Get the current working directory (cwd)
# files = os.listdir(cwd)  # Get all the files in that directory
# print("Files in %r: %s" % (cwd, files))

In [3]:
battles = pd.read_excel(
    '/Users/ryanbreier/RB.Data & Programming/Data Science/Projects/WWI Casualties/wiki_battles_html.xlsx'
)
battles['battle'] = battles['battle'].str.split('"').str[1]
battles_clean = battles.loc[battles['battle'].str.contains('/wiki/')]
battles_list = battles_clean['battle'].to_list()
battles_list = battles_list[0:5]  #Remove the '0:5' when done testing

In [4]:
data = pd.DataFrame([], columns=['battle','date','coordinates','side_1','side_2','cas_left','cas_right'])
for i in battles_list:
#     try:
        page = requests.get('https://en.wikipedia.org' + i)
        tree = html.fromstring(page.content)
        
        battle = tree.xpath('//*[@id="mw-content-text"]/div/..//tr[1]/th[1]/text()')
        date = tree.xpath('//*[text()="Date"]/..//td/text()')
        coord = tree.xpath('//span[@class="geo-dec"]/text()')  #Battle location coordinates
        
#         left = tree.xpath('//*[@id="mw-content-text"]/div/table[1]/..//tr[14]/td[1]/a/text()')
        fi = tree.xpath('//th[text()="Belligerents"]/parent::tr/following-sibling::tr[1]/td[1]/a/text()')
#         right = tree.xpath('//span[@class="datasortkey"]/..//a[1]/text()')
#         dsk = tree.xpath('//*[@class="datasortkey"]/ancestor::td//a[1]/text()')
        dsk = tree.xpath('//th[text()="Belligerents"]/parent::tr/following-sibling::tr[1]/td[2]/a/text()')
        
        casualties_fi = tree.xpath('//*[@class="infobox vevent"]/div/table[1]/tbody/tr[14]/td[*]/text()')
        casualties_dsk = tree.xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr[14]/td[*]/text()')
        data = data.append({'battle': battle[0],
                            'date': date[0],
                            'coordinates': coord[0],
                            'side_1': fi,
                            'side_2': dsk,
                            'cas_left': casualties_fi,
                            'cas_right': casualties_dsk,
                           },
                           ignore_index=True
                          )
#     except:
#         continue
data

,battle,date,coordinates,side_1,side_2,cas_left,cas_right
0,Battle of Le Cateau,26 August 1914,50.10417°N 3.54444°E,[German Empire],[],[],"[\n7,812 (700 killed, 2,600 captured), 38 guns]"
1,Battle of St Quentin/Battle of Guise,29–30 August 1914,49.90083°N 3.62833°E,[],[German Empire],[],[]
2,First Battle of the Marne,6–12 September 1914,49.017°N 3.383°E,[],[],[],[]
3,First Battle of the Aisne,13–28 September 1914,49.433°N 3.667°E,"[France, United Kingdom]",[Germany],[],[]
4,Siege of Antwerp,28 September – 10 October 1914\n(1 week and 5 ...,51.217°N 4.400°E,"[German Empire, Austria-Hungary]",[Belgium],[],[]
